In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed

# URL de la página con la lista de diputados
url_base = 'https://sitl.diputados.gob.mx/LXV_leg/listado_diputados_gpnp.php?tipot=TOTAL'

# Función para obtener el objeto BeautifulSoup de una página de forma concurrente
def obtener_html(url):
    with requests.Session() as session:  # Utiliza una sesión para reutilizar la conexión
        respuesta = session.get(url, timeout=10)
        respuesta.raise_for_status()  # Lanzar error si la solicitud falla
        return BeautifulSoup(respuesta.content, 'html.parser')

# Función para extraer la lista de diputados y sus URLs
def extraer_lista_diputados(soup):
    diputados = []
    for enlace in soup.select('a.Estilolinks'):
        nombre = enlace.get_text(strip=True)
        url_diputado = 'https://sitl.diputados.gob.mx/LXV_leg/' + enlace['href']
        diputados.append({'nombre': nombre, 'url': url_diputado})
    return diputados

# Función para extraer la información educativa de la página de un diputado
def extraer_informacion_educativa(soup):
    educacion = []
    # Encuentra todas las entradas después del encabezado 'ESCOLARIDAD'
    seccion_escolaridad = soup.find('td', class_='TitulosVerde', text='ESCOLARIDAD')
    if seccion_escolaridad:
        # La sección de escolaridad está en la misma tabla, pero después del título 'ESCOLARIDAD'
        entradas_escolaridad = seccion_escolaridad.find_all_next('tr')
        for entrada in entradas_escolaridad:
            celdas = entrada.find_all('td', class_='textoNegro')
            # Podría haber filas que no pertenecen a la sección escolaridad, como 'TRAYECTORIA POLÍTICA'
            # Así que verifica si la fila pertenece a la sección escolaridad
            if celdas and len(celdas) >= 2:  # Asumiendo que hay al menos dos celdas (grado y campo)
                # Extraer el grado, campo y periodo si están presentes
                grado = celdas[0].get_text(strip=True) if len(celdas) > 0 else ''
                campo = celdas[1].get_text(strip=True) if len(celdas) > 1 else ''
                periodo = celdas[2].get_text(strip=True) if len(celdas) > 2 else ''
                educacion.append({'grado': grado, 'campo': campo, 'periodo': periodo})
            # Si se encuentra un nuevo encabezado, se detiene la extracción
            if entrada.find('td', class_='TitulosVerde'):
                break
    return educacion

# Función para extraer los enlaces a las iniciativas de un diputado
def extraer_enlaces_iniciativas(soup):
    enlace_iniciativas = soup.find('a', text='Iniciativas')
    if enlace_iniciativas:
        url_iniciativas = 'https://sitl.diputados.gob.mx/LXV_leg/' + enlace_iniciativas['href']
        soup_iniciativas = obtener_html(url_iniciativas)
        enlaces_periodos = soup_iniciativas.select('a.estilolinks')
        iniciativas = [extraer_texto_iniciativas('https://sitl.diputados.gob.mx/LXV_leg/' + enlace['href']) for enlace in enlaces_periodos]
        return iniciativas
    return []

# Función para extraer el texto de las iniciativas de un periodo de sesiones
def extraer_texto_iniciativas(url_periodo):
    soup_periodo = obtener_html(url_periodo)
    textos = []
    for iniciativa in soup_periodo.select('span.Estiloparrafoc'):
        texto = iniciativa.get_text(strip=True)
        if texto:
            textos.append(texto)
    return textos

# Función para obtener la información de un diputado
def obtener_info_diputado(url):
    soup_diputado = obtener_html(url)
    nombre_element = soup_diputado.find(lambda tag: tag.name == "td" and tag.get("height") == "23" and tag.strong)
    if nombre_element and nombre_element.strong:
        nombre = nombre_element.strong.text.strip()
    else:
        nombre = "Nombre no encontrado"
    educacion = extraer_informacion_educativa(soup_diputado)
    iniciativas = extraer_enlaces_iniciativas(soup_diputado)
    return {
        'nombre': nombre,
        'educación': educacion,
        'iniciativas': iniciativas
    }

# Función principal para extraer la información de todos los diputados
def extraer_informacion_diputados_concurrent():
    soup_lista = obtener_html(url_base)
    lista_diputados = extraer_lista_diputados(soup_lista)
    urls_diputados = [diputado['url'] for diputado in lista_diputados]
    
    datos_diputados = []
    with ThreadPoolExecutor(max_workers=10) as executor:
        future_to_url = {executor.submit(obtener_info_diputado, url): url for url in urls_diputados}
        for future in as_completed(future_to_url):
            try:
                datos_diputados.append(future.result())
            except Exception as exc:
                print(f'Hubo un problema con la URL {future_to_url[future]}: {exc}')
    
    return pd.DataFrame(datos_diputados)

# Ejecutar la función principal y mostrar/guardar el DataFrame resultante
df_diputados = extraer_informacion_diputados_concurrent()
print(df_diputados)

                                      nombre  \
0           Dip. Odette Nayeri Almazán Muñoz   
1           Dip. Rosa Maria Alvarado Murguía   
2          Dip. Karla Yuritzi Almazán Burgos   
3        Dip. Martha Alicia Arreola Martinez   
4              Dip. Carol Antonio Altamirano   
..                                       ...   
495           Dip. Laura Lynn Fernández Piña   
496  Dip. Luis Ángel Xariel Espinosa Cházaro   
497                       Dip. Gabriela Sodi   
498              Dip. Susana Prieto Terrazas   
499     Dip. Claudia Gabriela Olvera Higuera   

                                             educacion  \
0                                                   []   
1                                                   []   
2    [{'grado': 'Diplomado', 'campo': 'Ciencias Pol...   
3                                                   []   
4    [{'grado': 'Licenciatura', 'campo': 'Derecho',...   
..                                                 ...   
495  [{'grado': '

In [3]:
# Suponiendo que 'df_diputados' es tu DataFrame

# Convertir el DataFrame a CSV
nombre_archivo = "diputados_1.5.csv"
df_diputados.to_csv(nombre_archivo, index=False, encoding='utf-8-sig')

print(f"Archivo guardado como: {nombre_archivo}")

Archivo guardado como: diputados_1.5.csv


In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

# URL de la página con la lista de diputados
url_base = 'https://sitl.diputados.gob.mx/LXV_leg/listado_diputados_gpnp.php?tipot=TOTAL'

# Configurar la sesión con reintentos
session = requests.Session()
retries = Retry(total=5, backoff_factor=1, status_forcelist=[502, 503, 504])
session.mount('https://', HTTPAdapter(max_retries=retries))

# Función para obtener el objeto BeautifulSoup de una página de forma concurrente
def obtener_html(url):
    respuesta = session.get(url, timeout=5)
    respuesta.raise_for_status()
    return BeautifulSoup(respuesta.content, 'html.parser')

# Función para extraer la lista de diputados y sus URLs
def extraer_lista_diputados(soup):
    diputados = []
    for enlace in soup.select('a.Estilolinks'):
        nombre = enlace.get_text(strip=True)
        url_diputado = 'https://sitl.diputados.gob.mx/LXV_leg/' + enlace['href']
        diputados.append({'nombre': nombre, 'url': url_diputado})
    return diputados

# Función ajustada para extraer correctamente la entidad del diputado
def extraer_entidad(soup):
    entidad_texto = soup.find(text="Entidad:")
    entidad = entidad_texto.find_next().text.strip() if entidad_texto else "Entidad no encontrada"
    entidad = entidad.split('\n')[0]  # Solo toma la primera línea que es el nombre de la entidad
    return entidad

# Función para extraer la información educativa de la página de un diputado
def extraer_informacion_educativa(soup):
    educacion = []
    # Encuentra todas las entradas después del encabezado 'ESCOLARIDAD'
    seccion_escolaridad = soup.find('td', class_='TitulosVerde', text='ESCOLARIDAD')
    if seccion_escolaridad:
        # La sección de escolaridad está en la misma tabla, pero después del título 'ESCOLARIDAD'
        entradas_escolaridad = seccion_escolaridad.find_all_next('tr')
        for entrada in entradas_escolaridad:
            celdas = entrada.find_all('td', class_='textoNegro')
            # Podría haber filas que no pertenecen a la sección escolaridad, como 'TRAYECTORIA POLÍTICA'
            # Así que verifica si la fila pertenece a la sección escolaridad
            if celdas and len(celdas) >= 2:  # Asumiendo que hay al menos dos celdas (grado y campo)
                # Extraer el grado, campo y periodo si están presentes
                grado = celdas[0].get_text(strip=True) if len(celdas) > 0 else ''
                campo = celdas[1].get_text(strip=True) if len(celdas) > 1 else ''
                periodo = celdas[2].get_text(strip=True) if len(celdas) > 2 else ''
                educacion.append({'grado': grado, 'campo': campo, 'periodo': periodo})
            # Si se encuentra un nuevo encabezado, se detiene la extracción
            if entrada.find('td', class_='TitulosVerde'):
                break
    return educacion

# Función para extraer los enlaces a las iniciativas de un diputado
def extraer_enlaces_iniciativas(soup):
    enlace_iniciativas = soup.find('a', text='Iniciativas')
    if enlace_iniciativas:
        url_iniciativas = 'https://sitl.diputados.gob.mx/LXV_leg/' + enlace_iniciativas['href']
        soup_iniciativas = obtener_html(url_iniciativas)
        enlaces_periodos = soup_iniciativas.select('a.estilolinks')
        iniciativas = [extraer_texto_iniciativas('https://sitl.diputados.gob.mx/LXV_leg/' + enlace['href']) for enlace in enlaces_periodos]
        return iniciativas
    return []

# Función para extraer el texto de las iniciativas de un periodo de sesiones
def extraer_texto_iniciativas(url_periodo):
    soup_periodo = obtener_html(url_periodo)
    textos = []
    for iniciativa in soup_periodo.select('span.Estiloparrafoc'):
        texto = iniciativa.get_text(strip=True)
        if texto:
            textos.append(texto)
    return textos

# Función para obtener la información de un diputado
def obtener_info_diputado(url):
    try:
        soup_diputado = obtener_html(url)
        nombre_element = soup_diputado.find(lambda tag: tag.name == "font" and "Dip." in tag.text)
        nombre = nombre_element.text.strip() if nombre_element else "Nombre no encontrado"
        entidad = extraer_entidad(soup_diputado)
        educacion = extraer_informacion_educativa(soup_diputado)
        iniciativas = extraer_enlaces_iniciativas(soup_diputado)
        return {
            'nombre': nombre,
            'entidad': entidad,
            'educación': educacion,
            'iniciativas': iniciativas
        }
    except Exception as e:
        print(f'Error al procesar {url}: {e}')
        return None  # Devolver None en caso de error

# Función principal para extraer la información de todos los diputados
def extraer_informacion_diputados_concurrent():
    soup_lista = obtener_html(url_base)
    lista_diputados = extraer_lista_diputados(soup_lista)
    urls_diputados = [diputado['url'] for diputado in lista_diputados]

    datos_diputados = []
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = [executor.submit(obtener_info_diputado, url) for url in urls_diputados]
        for future in as_completed(futures):
            result = future.result()
            if result:
                datos_diputados.append(result)

    return pd.DataFrame(datos_diputados)

# Ejecutar la función principal y mostrar/guardar el DataFrame resultante
df_diputados = extraer_informacion_diputados_concurrent()
print(df_diputados)

                                       nombre                entidad  \
0    Dip. Carol Antonio Altamirano (LICENCIA)                Oaxaca    
1            Dip. Odette Nayeri Almazán Muñoz                Puebla    
2            Dip. Maria Isabel Alfaro Morales               Hidalgo    
3         Dip. Martha Alicia Arreola Martinez               Durango    
4           Dip. Karla Yuritzi Almazán Burgos                México    
..                                        ...                    ...   
495            Dip. Laura Lynn Fernández Piña          Quintana Roo    
496                        Dip. Gabriela Sodi      Ciudad de México    
497   Dip. Luis Ángel Xariel Espinosa Cházaro  Entidad no encontrada   
498               Dip. Susana Prieto Terrazas  Entidad no encontrada   
499      Dip. Claudia Gabriela Olvera Higuera                México    

                                             educación  \
0    [{'grado': 'Licenciatura', 'campo': 'Derecho',...   
1                  

In [3]:
# Suponiendo que 'df_diputados' es tu DataFrame

# Convertir el DataFrame a CSV
nombre_archivo = "diputados_1.5.csv"
df_diputados.to_csv(nombre_archivo, index=False, encoding='utf-8-sig')

print(f"Archivo guardado como: {nombre_archivo}")

Archivo guardado como: diputados_1.5.csv
